1. 문서의 내용을 읽는다. (docx2txt, langchain-community)
2. 문서를 쪼갠다.
    - 토큰 수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [61]:
# pip install -qU docx2txt langchain-community

In [62]:
# docx 파일을 텍스트로 불러오는 역할
from langchain_community.document_loaders import Docx2txtLoader

# 불러오는 l length 의 텍스트 기반 리스트를 잘라주는 역할
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(

    # 문서를 쪼갤 때 얼마나 많은 토큰 (단어) 수마다 자를지 결정
    chunk_size=1500,

    # 문맥을 파악하기 위해 다음 자름 포인트보다 overlap 만큼 더 앞 부분부터 자름
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')

# 이렇게 가져오면 통 리스트 하나로 가져올 수 있음
# document = loader.load()

# 이렇게 가져오면 `text_splitter` 로 자른 사이즈들이 리스트데 담겨서 가져와짐
document_list = loader.load_and_split(text_splitter=text_splitter)

# 이렇게 불러오면 하나의 리스트 형태로 불러와진다. (length 1)
# 이걸 쪼개야 하는데, 여기서 text-splitter 가 필요하다.
# pip install -qU langchain-text-splitters

In [63]:
from dotenv import load_dotenv

# 쪼갠 문서를 임베딩한다.
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [64]:
# 벡터 데이터베이스인 Chroma 를 사용한다.
# %pip install langchain-chroma

from langchain_chroma import Chroma

# 위에서 쪼개놓은 문서를 기반으로 백터 데이터베이스에 저장한다.
database = Chroma.from_documents(

    # 문서
    documents=document_list, 

    # 임베드 객체
    embedding=embedding,

    # 히스토리를 저장할 DB
    persist_directory='./Chroma',

    # DB Name
    collection_name='chroma-tax'
    )

In [65]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# 백터 DB 에서 유사도 검색을 한다.
# 위에 넣었던 임베딩을 활용해서 알아서 답변을 가져온다.
retrieved_docs = database.similarity_search(query)

In [66]:
retrieved_docs

[Document(metadata={'source': './tax.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제18호 및 제21호에 따른 기타소득에 대해서는 100분의 15\n\n다. 삭제<2014. 12. 2

In [67]:
# 문서를 가져왔으니, 이제 LLM 에 질의를 해야 한다.
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

promt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [69]:
ai_message = llm.invoke(promt)

In [70]:
ai_message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 근로소득세율과 공제를 적용해야 합니다. 일반적으로 소득세는 과세표준에 따라 누진세율을 적용하며, 공제 항목에 따라 다소 차이가 있을 수 있습니다. 아래는 대략적인 계산 과정을 설명합니다.\n\n1. **총급여액**: 5천만원\n\n2. **근로소득공제**: 근로소득공제는 총급여액에 따라 다르게 적용됩니다. 대략적으로 계산하면 다음과 같습니다.\n   - 5천만원 × 30% = 약 1,500만원 (실제 공제율은 총급여액에 따라 세분화되어 있음)\n\n3. **과세표준**: 총급여에서 근로소득공제를 뺀 금액\n   - 5천만원 - 1,500만원 = 3,500만원\n\n4. **누진세율 적용**: 과세표준에 따라 소득세율을 적용합니다. 대략적으로 3천만원 이상의 과세표준에 대해서는 24%의 세율이 적용됩니다. 하지만 1,200만원 ~ 4,600만원까지는 15% 세율이 적용됩니다.\n   - 3,500만원에 대한 세율을 약 15%로 적용\n   - 3,500만원 × 15% = 약 525만원\n\n5. **세액공제**: 근로소득자에게는 일정 금액의 세액공제가 적용됩니다. 기본 세액공제가 약 50만원 정도 적용될 수 있습니다.\n   - 525만원 - 50만원 = 475만원\n\n최종적으로, 연봉 5천만원인 직장인의 소득세는 대략 475만원 정도로 추정할 수 있습니다. 이는 각종 세액공제 및 기타 개인 상황에 따라 변동될 수 있습니다. 보다 정확한 계산을 위해서는 국세청의 연말정산 간소화 서비스 등을 참고하는 것이 좋습니다.'